Begin with the common imports, `Dataset` and `URL`. As these two are always used, they are available from `remotemanager` root.

In [1]:
from remotemanager import Dataset, URL
from remotemanager.serialisation import serialjson

Define a simple function to run:

In [2]:
def basic_function(inp):
    import time
    
    time.sleep(1)
    
    return inp*inp

For the moment, we're just running on our local machines

In [3]:
url = URL()
# url = URL(host='vega')

Basic setup done, lets create the Dataset

In [4]:
ds = Dataset(function=basic_function,
             url=url,
             # script='module load Python/3.10.4-GCCcore-11.3.0',  # not changing the script to load the python module causes issues
             # serialiser=serialjson(),  # needs updating as yaml isn't available
            )

The Dataset stores the _function_, the Runners store the _arguments_

Right now all we have is a function, need to create the args:

In [5]:
values = [1, 3, 7, 50]

for val in values:
    ds.append_run(args={'inp': val})

Now we have all the material required:

- Function
- Connection
- Arguments

Time to run:

In [6]:
ds.run(force=True)

The below cell is useful to wait on a run function, there are two sections to it:

### `print(ds.run_cmds)`

This checks the commands that were used to launch the command were okay. If there was any errors, you'll see them here.

### `while not ds.all_finished: ...`

This block waits for the dataset to be completed. `Dataset.all_finished` only returns `True` when all the runners are completed.

Note: You can also use `Dataset.is_finished` to see the state on a per-runner basis.

In [7]:
print(ds.run_cmds)

import time
while not ds.all_finished:
    print('dataset not finished yet, sleeping for 1s')
    time.sleep(1)

[]


If we've made it through the wait block then we must have results, lets fetch them:

In [11]:
ds.fetch_results()

Now they're fetched, we can access them via the `results` property without having to talk to the remote again:

In [12]:
ds.results

[1, 9, 49, 2500]

In [13]:
ds.runners[0].history

{'2022-11-07 13:48:48/0': 'created',
 '2022-11-07 13:48:48/1': 'submitted',
 '2022-11-07 13:48:49/0': 'resultfile created remotely',
 '2022-11-07 13:48:49/1': 'completed',
 '2022-11-07 13:50:14/0': 'submitted',
 '2022-11-07 13:52:16/0': 'submitted',
 '2022-11-07 13:52:34/0': 'submitted'}